In [1]:
import spacy

In [ ]:
!pip install en_core_web_sm
!pip install spacy nltk scikit-learn
!pip install sentence-transformers

In [28]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp("corona")
doc2=nlp("I have headache and something like flu but it can be also related to virus")
doc3=nlp("I went to doctor for pcr test, i hope i dont have any covid")

In [47]:
print(doc1.similarity(doc2))
print(doc1.similarity(doc3))

0.03827971584133683
0.21223290349249319


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. 

In [30]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
cosine_sim("corona","I have headache and something like flu but it can be also related to virus")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  % sorted(

0.0

In [32]:
cosine_sim("corona","I went to doctor for pcr test, i hope i dont have any covid")

0.0

In [33]:
corpus=["corona","I have headache and something like flu but it can be also related to virus","I went to doctor for pcr test, i hope i dont have any covid"]
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T 

In [34]:
pairwise_similarity.toarray()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [41]:
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('all-MiniLM-L6-v2')

paragraph_emb1 = model.encode("Nanoparticle (NP)–protein complexes exhibit the “correct identity” of NP in biological media. Therefore, protein–NP interactions should be closely explored to understand and modulate the nature of NPs in medical implementations. This review focuses mainly on the physicochemical parameters such as dimension, surface chemistry, morphology of NPs, and influence of pH on the formation of protein corona and conformational changes of adsorbed proteins by different kinds of techniques. Also, the impact of protein corona on the colloidal stability of NPs is discussed. Uncontrolled protein attachment on NPs may bring unwanted impacts such as protein denaturation and aggregation. In contrast, controlled protein adsorption by optimal concentration, size, pH, and surface modification of NPs may result in potential implementation of NPs as therapeutic agents especially for disaggregation of amyloid fibrils. Also, the effect of NPs-protein corona on reducing the cytotoxicity and clinical implications such as drug delivery, cancer therapy, imaging and diagnosis will be discussed. Validated correlative physicochemical parameters for NP–protein corona formation frequently derived from protein corona fingerprints of NPs which are more valid than the parameters obtained only on the base of NP features. This review may provide useful information regarding the potency as well as the adverse effects of NPs to predict their behavior in vivo.")
query_emb = model.encode("corona")
sim_score = util.pytorch_cos_sim(query_emb, paragraph_emb1)
print(sim_score)

tensor([[0.2003]])


In [42]:
paragraph_emb = model.encode("Nanoparticles (NPs) in contact with a biological medium are rapidly comprehended by a number of protein molecules resulting in the formation of an NP–protein complex called protein corona (PC). The cell sees the protein-coated NPs as the synthetic identity is masked by protein surfacing. The PC formation ultimately has a substantial impact on various biological processes including drug release, drug targeting, cell recognition, biodistribution, cellular uptake, and therapeutic efficacy. Further, the composition of PC is largely influenced by the physico-chemical properties of NPs viz. the size, shape, surface charge, and surface chemistry in the biological milieu. However, the change in the biological responses of the new substrate depends on the quantity of protein access by the NPs. The PC-layered NPs act as new biological entities and are recognized as different targeting agents for the receptor-mediated ingress of therapeutics in the biological cells. The corona-enveloped NPs have both pros and cons in the biological system. The review provides a brief insight into the impact of biomolecules on nanomaterials carrying cargos and their ultimate fate in the biological milieu.")
query_emb = model.encode("corona")
sim_score = util.pytorch_cos_sim(query_emb, paragraph_emb)
print(sim_score)

tensor([[0.2250]])
